In [64]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [65]:
CSV_COLUMN_NAMES = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality' ]
QUALITY_GROUPS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [66]:
df_train=pd.read_csv('winequality_train.csv', delimiter=";")
df_eval=pd.read_csv('winequality_eval.csv', delimiter=";")

In [67]:
df_train.head()
df_eval.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.5,0.36,0.31,13.55,0.053,20,113,0.99544,3.20,0.56,11.0,6
1,6.5,0.16,0.33,1.00,0.027,23,75,0.99080,3.30,0.39,11.4,7
2,8.3,0.22,0.34,1.10,0.043,20,116,0.99270,3.00,0.47,10.2,6
3,6.9,0.23,0.35,6.90,0.030,45,116,0.99244,2.80,0.54,11.0,6
4,6.4,0.17,0.34,13.40,0.044,45,139,0.99752,3.06,0.43,9.1,6


In [68]:
y_train=df_train.pop('quality')
y_eval=df_eval.pop('quality')
df_train.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [69]:
df_train.shape

(4598, 11)

In [70]:
def input_fn(features, labels, training=True, batch_size=64):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [71]:
my_feature_columns = []
for key in df_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='fixed acidity', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='volatile acidity', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='citric acid', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='residual sugar', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='chlorides', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='free sulfur dioxide', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='total sulfur dioxide', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='density', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='pH', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='sulphates', shape=(1,), default_

In [72]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns, hidden_units=[10, 10, 10], n_classes=10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Paulina\\AppData\\Local\\Temp\\tmpnr7j7x0n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [73]:
classifier.train(
    input_fn=lambda: input_fn(df_train, y_train, training=True),
    steps=20000)

INFO:tensorflow:Calling model_fn.


ValueError: in user code:

    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_estimator\python\estimator\canned\dnn.py:350 call  *
        net = self._input_layer(features, training=is_training)
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:757 __call__  **
        self._maybe_build(inputs)
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:2098 _maybe_build
        self.build(input_shapes)
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\feature_column\dense_features_v2.py:92 build
        with ops.name_scope(column.name):
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\framework\ops.py:6492 __enter__
        return self._name_scope.__enter__()
    C:\Users\Paulina\AppData\Local\Programs\Python\P

In [74]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(df_eval, y_eval, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Could not find trained model in model_dir: C:\Users\Paulina\AppData\Local\Temp\tmpnr7j7x0n, running initialization to evaluate.
INFO:tensorflow:Calling model_fn.


ValueError: in user code:

    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_estimator\python\estimator\canned\dnn.py:350 call  *
        net = self._input_layer(features, training=is_training)
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:757 __call__  **
        self._maybe_build(inputs)
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:2098 _maybe_build
        self.build(input_shapes)
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\feature_column\dense_features_v2.py:92 build
        with ops.name_scope(column.name):
    C:\Users\Paulina\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\framework\ops.py:6492 __enter__
        return self._name_scope.__enter__()
    C:\Users\Paulina\AppData\Local\Programs\Python\P